
# WA Conservation & Sensitive Lists

### List Identifiers
* Conservation list: __[dr2201](https://lists.ala.org.au/speciesListItem/list/dr2201)__ (and [dr2201 in test](https://lists-test.ala.org.au/speciesListItem/list/dr2201))
* Sensitive list: __[dr467](https://lists.ala.org.au/speciesListItem/list/dr467)__ (and [dr18406 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18406))

### Source Data

* __[WA Dept of Biodiversity, Conservation and Attractions, Parks and Wildlife, Threatened Species Information](https://www.dpaw.wa.gov.au/plants-and-animals/threatened-species-and-communities)__
* __[Threatened and Priority Fauna List 7 October 2022.xlsx](https://www.dpaw.wa.gov.au/images/Threatened%20and%20Priority%20Fauna%20List%207%20October%202022.xlsx)__
* __[Threatened and Priority Flora List 6 October 2022.xlsx](https://www.dpaw.wa.gov.au/images/documents/plants-animals/threatened-species/Listings/Threatened%20and%20Priority%20Flora%20List_6%20October%202022.xlsx)__
* __[Conservation Codes Listing](https://www.dpaw.wa.gov.au/images/Conservation%20codes%20for%20Western%20Australian%20Flora%20and%20Fauna%20179KB.pdf)__

**Metadata Description**
The Biodiversity Conservation Act 2016 provides for the listing of threatened native plants (flora), threatened native animals (fauna) and threatened ecological communities that need protection as critically endangered, endangered or vulnerable species or ecological communities because they are under identifiable threat of extinction (species) or collapse (ecological communities).

The Western Australian Threatened Species Scientific Committee and the Threatened Ecological Communities Scientific Committee review nominations to list, delist or change the threat status of threatened species and ecological communities, and use the International Union for Conservation of Nature Red List criteria for assigning species and communities to threat categories.

This list is a concatenation of the WA flora and fauna state lists which have been downloaded from https://www.dpaw.wa.gov.au/images/documents/plants-animals/threatened-species/Listings/Threatened%20and%20Priority%20Flora%20List_6%20October%202022.xlsx and https://www.dpaw.wa.gov.au/images/Threatened%20and%20Priority%20Fauna%20List%207%20October%202022.xlsx and parsed for use in the Atlas of Living Australia.

**Metadata URL**
https://www.dpaw.wa.gov.au/plants-and-animals/threatened-species-and-communities


**Note:**
* WA uses the same data files for both conservation and sensitive lists.
* Fauna and Flora are in separate files and have different formats.
* Files are processed separately and then concatenated into a single file.

## 1. Prep
Import libraries
Set project directories
Download files if necessary

In [23]:
import pandas as pd
import numpy as np
import requests

projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sourceDataDir = projectDir + "source-data/WA/"
processedDataDir = projectDir + "current-lists/"

faunafilename = sourceDataDir + "Threatened and Priority Fauna List 7 October2022.csv"
florafilename = sourceDataDir + "Threatened and Priority Flora List 6 October2022.csv"

In [24]:
%%script echo skipping # comment this line to download the files and save locally

print("Downloading fauna file to:" + faunafilename)
faunaurl = "https://www.dpaw.wa.gov.au/images/Threatened%20and%20Priority%20Fauna%20List%207%20October%202022.xlsx"
c = requests.get(faunaurl).content
fauna = pd.read_excel(c)
fauna.to_csv(faunafilename)

print("Downloading flora file to:" + florafilename)
floraurl = "https://www.dpaw.wa.gov.au/images/documents/plants-animals/threatened-species/Listings/Threatened%20and%20Priority%20Flora%20List_6%20October%202022.xlsx"
c = requests.get(floraurl).content
flora = pd.read_excel(c)
flora.to_csv(florafilename)

skipping # comment this line to download the files and save locally


## 2. WA Fauna

In [25]:
fauna = pd.read_csv(faunafilename)
fauna.head(3)

,Unnamed: 0,Scientific name,Common name,Class,WA listing,National listing,Kimberley,Pilbara,Goldfields,Midwest,Wheatbelt,South Coast,Swan,South West,Warren,WA listing.1,Notes
0,0,Abebaioscia troglodytes,Pannikin Plain Cave isopod,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN
1,1,Acanthophis antarcticus,southern death adder,REPTILE,P3,NaN,NaN,NaN,X,NaN,X,X,X,NaN,NaN,DBCA Priority species.,NaN
2,2,Acercella poorginup,Poorginup Swamp watermite,INVERTEBRATE,P2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,DBCA Priority species.,NaN


Summarise locality fields
‘X’ is marked in locality where the species is found - put into a single field

In [26]:
# summarise the locality
df = fauna.iloc[:,6:15].copy()
v = np.where(df == "X")
fauna['locality'] = (pd.DataFrame(df.columns[v[1]], index=df.index[v[0]], columns=['result']).groupby(level=0).agg(','.join).reindex(df.index))
fauna.iloc[:,[6,7,8,9,10,11,12,13,14,-1]].drop_duplicates() # just to demonstrate

,Kimberley,Pilbara,Goldfields,Midwest,Wheatbelt,South Coast,Swan,South West,Warren,locality
0,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,South Coast
1,NaN,NaN,X,NaN,X,X,X,NaN,NaN,"Goldfields,Wheatbelt,South Coast,Swan"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,Warren
5,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kimberley
6,X,X,X,X,X,X,X,X,X,"Kimberley,Pilbara,Goldfields,Midwest,Wheatbelt..."
...,...,...,...,...,...,...,...,...,...,...
484,NaN,NaN,X,NaN,X,X,NaN,NaN,NaN,"Goldfields,Wheatbelt,South Coast"
505,X,X,NaN,NaN,NaN,NaN,X,NaN,NaN,"Kimberley,Pilbara,Swan"
511,NaN,NaN,NaN,X,NaN,X,NaN,X,X,"Midwest,South Coast,South West,Warren"
512,X,NaN,NaN,NaN,X,NaN,X,NaN,NaN,"Kimberley,Wheatbelt,Swan"


Map status values

In [27]:
conservationCodesMapping = {
    'T':'Threatened',
    'CR':'Critically Endangered',
    'EN':'Endangered',
    'VU':'Vulnerable',
    'EX':'Extinct',
    'EW':'Extinct in the Wild',
    'SP':'Specially Protected',
    'MI':'Migratory',
    'CD':'Conservation Dependent',
    'CD & MI':'Conservation Dependent',
    'OS':'Other Specially Protected',
    'P': 'Priority',
    'P1':'Priority 1: Poorly-known species',
    'P2':'Priority 2: Poorly-known species',
    'P3':'Priority 3: Poorly-known species',
    'P4':'Priority 4: Rare, Near Threatened',
    'MI & P1':'Priority 1: Poorly-known species',
    'MI & P3':'Priority 2: Poorly-known species',
    'MI & P4':'Priority 4: Rare, Near Threatened'
}
statusLongDescriptionMapping = {
    'Priority 1: Poorly-known species':'Priority 1: Poorly-known species - known from few locations, none on conservation lands',
    'Priority 2: Poorly-known species':'Priority 2: Poorly-known species - known from few locations, some on conservation lands',
    'Priority 3: Poorly-known species':'Priority 3: Poorly-known species - known from several locations',
    'Priority 4: Rare, Near Threatened':'Priority 4: Rare, Near Threatened and other species in need of monitoring'
}

fauna['status'] = fauna['WA listing'].str.strip().map(conservationCodesMapping).fillna(fauna['WA listing'])
fauna['status description'] = fauna['status'].map(statusLongDescriptionMapping).fillna(fauna['status'])
fauna[['WA listing','status','status description']].drop_duplicates()

,WA listing,status,status description
0,VU,Vulnerable,Vulnerable
1,P3,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
2,P2,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
5,MI,Migratory,Migratory
7,CR,Critically Endangered,Critically Endangered
10,P1,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...
14,P4,"Priority 4: Rare, Near Threatened","Priority 4: Rare, Near Threatened and other sp..."
27,EN,Endangered,Endangered
40,OS,Other Specially Protected,Other Specially Protected
84,EX,Extinct,Extinct


## 3. WA Flora

In [28]:
flora = pd.read_csv(florafilename)
flora.head(3)

,Unnamed: 0,Name ID,Taxon,WA Status,WA Rank,WA IUCN Criteria,EPBC,DBCA Region,DBCA District,Distribution,Flowering Period,Recovery Plan,Notes
0,0,50593,Abildgaardia pachyptera,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Mitchell Plateau,Feb-Mar,NaN,NaN
1,1,14112,Abutilon sp. Hamelin (A.M. Ashby 2196),2,NaN,NaN,NaN,MWST,SHARK BAY,"Shark Bay, Hamelin Pool, Yaringa Stn",Jul-Sep,NaN,NaN
2,2,14110,Abutilon sp. Onslow (F. Smith s.n. 10/9/61),3,NaN,NaN,NaN,PILB,KARRATHA,"Onslow, Yaraloola Stn",Sep,NaN,NaN


Map status values

In [29]:
flora[[' WA Status','WA Rank']].drop_duplicates()

,WA Status,WA Rank
0,1,NaN
1,2,NaN
2,3,NaN
8,4,NaN
16,T,VU
20,T,CR
23,T,EN
131,X,EX


In [30]:
def setflorastatus(row):
    status = row[' WA Status']
    rank = row['WA Rank']
    s = "Unknown"
    if status == 'T':
        if rank == 'VU':
            s = "Vulnerable"
        elif rank == 'CR':
            s = "Critically Endangered"
        elif rank == 'EN':
            s = "Endangered"
    elif status == 'X':
        s = "Extinct"
    elif status == '1':
        s = 'Priority 1: Poorly-known species'
    elif status == '2':
        s = 'Priority 2: Poorly-known species'
    elif status == '3':
        s = 'Priority 3: Poorly-known species'
    elif status == '4':
        s = 'Priority 4: Rare, Near Threatened'
    return s
flora = flora.assign(status = flora.apply(setflorastatus,axis=1))
flora['status description'] = flora['status'].map(statusLongDescriptionMapping).fillna(flora['status'])
flora[[' WA Status','WA Rank','status','status description']].drop_duplicates()

,WA Status,WA Rank,status,status description
0,1,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...
1,2,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
2,3,NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
8,4,NaN,"Priority 4: Rare, Near Threatened","Priority 4: Rare, Near Threatened and other sp..."
16,T,VU,Vulnerable,Vulnerable
20,T,CR,Critically Endangered,Critically Endangered
23,T,EN,Endangered,Endangered
131,X,EX,Extinct,Extinct


In [31]:

flora

,Unnamed: 0,Name ID,Taxon,WA Status,WA Rank,WA IUCN Criteria,EPBC,DBCA Region,DBCA District,Distribution,Flowering Period,Recovery Plan,Notes,status,status description
0,0,50593,Abildgaardia pachyptera,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Mitchell Plateau,Feb-Mar,NaN,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...
1,1,14112,Abutilon sp. Hamelin (A.M. Ashby 2196),2,NaN,NaN,NaN,MWST,SHARK BAY,"Shark Bay, Hamelin Pool, Yaringa Stn",Jul-Sep,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
2,2,14110,Abutilon sp. Onslow (F. Smith s.n. 10/9/61),3,NaN,NaN,NaN,PILB,KARRATHA,"Onslow, Yaraloola Stn",Sep,NaN,NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
3,3,43021,Abutilon sp. Pritzelianum (S. van Leeuwen 5095),3,NaN,NaN,NaN,"MWST,PILB","KARRATHA,GERALDTON","Carnarvon, Port Headland, Lake McLeod, Gascoyn...","Jun, Aug-Sep",NaN,NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
4,4,14114,Abutilon sp. Quobba (H. Demarz 3858),2,NaN,NaN,NaN,"MWST,PILB","EXMOUTH,GERALDTON,SHARK BAY","Quobba, Cape Range, Muggon Stn., Canarvon, Dor...",Jul-Oct,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,3854,17481,Xyris maxima,2,NaN,NaN,NaN,SWST,BLACKWOOD,Witchcliffe,Nov-Jan,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
3855,3855,7383,Zehneria mucronata,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,"Mitchell Plateau, Cassini Is., Sale River",Jan,NaN,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...
3856,3856,29060,Zeuxine oblonga,2,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Kununurra,NaN,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
3857,3857,36237,Zornia areolata,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Kununurra,NaN,NaN,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...


## 3. Concatenate Flora and Fauna dataframes

In [32]:
flora

,Unnamed: 0,Name ID,Taxon,WA Status,WA Rank,WA IUCN Criteria,EPBC,DBCA Region,DBCA District,Distribution,Flowering Period,Recovery Plan,Notes,status,status description
0,0,50593,Abildgaardia pachyptera,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Mitchell Plateau,Feb-Mar,NaN,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...
1,1,14112,Abutilon sp. Hamelin (A.M. Ashby 2196),2,NaN,NaN,NaN,MWST,SHARK BAY,"Shark Bay, Hamelin Pool, Yaringa Stn",Jul-Sep,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
2,2,14110,Abutilon sp. Onslow (F. Smith s.n. 10/9/61),3,NaN,NaN,NaN,PILB,KARRATHA,"Onslow, Yaraloola Stn",Sep,NaN,NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
3,3,43021,Abutilon sp. Pritzelianum (S. van Leeuwen 5095),3,NaN,NaN,NaN,"MWST,PILB","KARRATHA,GERALDTON","Carnarvon, Port Headland, Lake McLeod, Gascoyn...","Jun, Aug-Sep",NaN,NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
4,4,14114,Abutilon sp. Quobba (H. Demarz 3858),2,NaN,NaN,NaN,"MWST,PILB","EXMOUTH,GERALDTON,SHARK BAY","Quobba, Cape Range, Muggon Stn., Canarvon, Dor...",Jul-Oct,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,3854,17481,Xyris maxima,2,NaN,NaN,NaN,SWST,BLACKWOOD,Witchcliffe,Nov-Jan,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
3855,3855,7383,Zehneria mucronata,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,"Mitchell Plateau, Cassini Is., Sale River",Jan,NaN,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...
3856,3856,29060,Zeuxine oblonga,2,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Kununurra,NaN,NaN,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
3857,3857,36237,Zornia areolata,1,NaN,NaN,NaN,KIMB,EAST KIMBERLEY,Kununurra,NaN,NaN,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...


In [33]:
fauna

,Unnamed: 0,Scientific name,Common name,Class,WA listing,National listing,Kimberley,Pilbara,Goldfields,Midwest,Wheatbelt,South Coast,Swan,South West,Warren,WA listing.1,Notes,locality,status,status description
0,0,Abebaioscia troglodytes,Pannikin Plain Cave isopod,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN,South Coast,Vulnerable,Vulnerable
1,1,Acanthophis antarcticus,southern death adder,REPTILE,P3,NaN,NaN,NaN,X,NaN,X,X,X,NaN,NaN,DBCA Priority species.,NaN,"Goldfields,Wheatbelt,South Coast,Swan",Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...
2,2,Acercella poorginup,Poorginup Swamp watermite,INVERTEBRATE,P2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,DBCA Priority species.,NaN,Warren,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...
3,3,Acizzia mccarthyi,McCarthy's plant-louse,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN,South Coast,Vulnerable,Vulnerable
4,4,Acizzia veski,Vesk's plant-louse,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN,South Coast,Vulnerable,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,591,Zephyrarchaea mainae,Main's assassin spider,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,X,Threatened species - Vulnerable under the Biod...,NaN,"South Coast,Warren",Vulnerable,Vulnerable
592,592,Zephyrarchaea marki,Cape Le Grand assassin spider,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN,South Coast,Vulnerable,Vulnerable
593,593,Zephyrarchaea melindae,Toolbrunup assassin spider,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN,South Coast,Vulnerable,Vulnerable
594,594,Zephyrarchaea robinsi,eastern massif assassin spider,INVERTEBRATE,VU,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,Threatened species - Vulnerable under the Biod...,NaN,South Coast,Vulnerable,Vulnerable


In [34]:
# line up the fields
flora = flora.drop(['Flowering Period', 'WA IUCN Criteria','Recovery Plan'], axis=1)
flora['sourceStatus'] = flora.apply(lambda x: x[' WA Status'] if pd.isnull(x['WA Rank']) else x['WA Rank'],axis=1)
flora['DBCA District'] = flora['DBCA District'].str.title()
flora = flora.rename(columns=
         {'Name ID': 'taxonId',
          'Taxon': 'scientificName',
          ' WA Status': 'WA Status',
          'EPBC': 'epbcStatus',
          'Notes': 'taxonRemarks'
          })
flora['taxonId'] = flora['taxonId'].astype(str)
fauna = fauna.rename(columns=
        {'Scientific name': 'scientificName',
         'Common name': 'vernacularName',
         'Class': 'class',
         'WA listing': 'sourceStatus',
         'National listing': 'epbcStatus',
         'Notes': 'taxonRemarks'
        })

In [35]:
flora

,Unnamed: 0,taxonId,scientificName,WA Status,WA Rank,epbcStatus,DBCA Region,DBCA District,Distribution,taxonRemarks,status,status description,sourceStatus
0,0,50593,Abildgaardia pachyptera,1,NaN,NaN,KIMB,East Kimberley,Mitchell Plateau,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...,1
1,1,14112,Abutilon sp. Hamelin (A.M. Ashby 2196),2,NaN,NaN,MWST,Shark Bay,"Shark Bay, Hamelin Pool, Yaringa Stn",NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...,2
2,2,14110,Abutilon sp. Onslow (F. Smith s.n. 10/9/61),3,NaN,NaN,PILB,Karratha,"Onslow, Yaraloola Stn",NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...,3
3,3,43021,Abutilon sp. Pritzelianum (S. van Leeuwen 5095),3,NaN,NaN,"MWST,PILB","Karratha,Geraldton","Carnarvon, Port Headland, Lake McLeod, Gascoyn...",NaN,Priority 3: Poorly-known species,Priority 3: Poorly-known species - known from ...,3
4,4,14114,Abutilon sp. Quobba (H. Demarz 3858),2,NaN,NaN,"MWST,PILB","Exmouth,Geraldton,Shark Bay","Quobba, Cape Range, Muggon Stn., Canarvon, Dor...",NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,3854,17481,Xyris maxima,2,NaN,NaN,SWST,Blackwood,Witchcliffe,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...,2
3855,3855,7383,Zehneria mucronata,1,NaN,NaN,KIMB,East Kimberley,"Mitchell Plateau, Cassini Is., Sale River",NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...,1
3856,3856,29060,Zeuxine oblonga,2,NaN,NaN,KIMB,East Kimberley,Kununurra,NaN,Priority 2: Poorly-known species,Priority 2: Poorly-known species - known from ...,2
3857,3857,36237,Zornia areolata,1,NaN,NaN,KIMB,East Kimberley,Kununurra,NaN,Priority 1: Poorly-known species,Priority 1: Poorly-known species - known from ...,1


In [36]:
#fauna = fauna[['scientificName','vernacularName','waListing','locality','status']]
#flora = flora[['taxonId','scientificName','sourceStatus','waRank','region']

In [37]:
floraandfauna = pd.concat([flora, fauna], axis=0, ignore_index=True, sort=False)
floraandfauna['sensitivityZoneId'] = "WA"
floraandfauna = floraandfauna[['taxonId','scientificName','vernacularName','WA Status','WA Rank','DBCA District','Distribution','taxonRemarks','status','sourceStatus','status description','class','sensitivityZoneId']]

In [38]:
floraandfauna

,taxonId,scientificName,vernacularName,WA Status,WA Rank,DBCA District,Distribution,taxonRemarks,status,sourceStatus,status description,class,sensitivityZoneId
0,50593,Abildgaardia pachyptera,NaN,1,NaN,East Kimberley,Mitchell Plateau,NaN,Priority 1: Poorly-known species,1,Priority 1: Poorly-known species - known from ...,NaN,WA
1,14112,Abutilon sp. Hamelin (A.M. Ashby 2196),NaN,2,NaN,Shark Bay,"Shark Bay, Hamelin Pool, Yaringa Stn",NaN,Priority 2: Poorly-known species,2,Priority 2: Poorly-known species - known from ...,NaN,WA
2,14110,Abutilon sp. Onslow (F. Smith s.n. 10/9/61),NaN,3,NaN,Karratha,"Onslow, Yaraloola Stn",NaN,Priority 3: Poorly-known species,3,Priority 3: Poorly-known species - known from ...,NaN,WA
3,43021,Abutilon sp. Pritzelianum (S. van Leeuwen 5095),NaN,3,NaN,"Karratha,Geraldton","Carnarvon, Port Headland, Lake McLeod, Gascoyn...",NaN,Priority 3: Poorly-known species,3,Priority 3: Poorly-known species - known from ...,NaN,WA
4,14114,Abutilon sp. Quobba (H. Demarz 3858),NaN,2,NaN,"Exmouth,Geraldton,Shark Bay","Quobba, Cape Range, Muggon Stn., Canarvon, Dor...",NaN,Priority 2: Poorly-known species,2,Priority 2: Poorly-known species - known from ...,NaN,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4450,NaN,Zephyrarchaea mainae,Main's assassin spider,NaN,NaN,NaN,NaN,NaN,Vulnerable,VU,Vulnerable,INVERTEBRATE,WA
4451,NaN,Zephyrarchaea marki,Cape Le Grand assassin spider,NaN,NaN,NaN,NaN,NaN,Vulnerable,VU,Vulnerable,INVERTEBRATE,WA
4452,NaN,Zephyrarchaea melindae,Toolbrunup assassin spider,NaN,NaN,NaN,NaN,NaN,Vulnerable,VU,Vulnerable,INVERTEBRATE,WA
4453,NaN,Zephyrarchaea robinsi,eastern massif assassin spider,NaN,NaN,NaN,NaN,NaN,Vulnerable,VU,Vulnerable,INVERTEBRATE,WA


Write dataframe to both conservation and sensitive CSV files - UTF-8 encoding

In [39]:
floraandfauna.to_csv(projectDir + "current-lists/conservation-lists/WA-conservation.csv", encoding="UTF-8",index=False)
floraandfauna.to_csv(projectDir + "current-lists/sensitive-lists/WA-sensitive.csv", encoding="UTF-8", index=False)

# Manual List check

* Conservation list: __[dr2201](https://lists.ala.org.au/speciesListItem/list/dr2201)__ (and [dr2201 in test](https://lists-test.ala.org.au/speciesListItem/list/dr2201))
* Sensitive list: __[dr467](https://lists.ala.org.au/speciesListItem/list/dr467)__ (and [dr18406 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18406))

**Instructions**
1. Load the lists above into the lists-test tool
2. Check the list name matching score and the text appearance on species pages
3. Unskip the below code and Run the reports below to compare to production. Send the changelog.csv to check. Correct any issues.
4. Save the production list into the `historical lists` directory by uncommenting the code section below.
5. Load the lists into production


In [45]:
# prep
import sys
import os
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf
import datetime
monthStr = datetime.datetime.now().strftime('%Y%m')

In [46]:
# %%script echo skipping # comment this line to do a list check between test and production and save the changelog
# conservation
filename = "WA-conservation.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr2201" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr2201" + "?max=10000&includeKVP=true"
changelist = lf.get_changelist(testListUrl, prodListUrl)
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

,name,commonName_new,scientificName_new,status_new,listUpdate,commonName_old,scientificName_old,status_old
0,Abildgaardia pachyptera,None,Abildgaardia pachyptera,Priority 1: Poorly-known species,added,NaN,NaN,NaN
60,Acacia corusca,Shimmer Wattle,Acacia corusca,Priority 1: Poorly-known species,added,NaN,NaN,NaN
126,Acacia karinae,Mulga,Acacia,Priority 1: Poorly-known species,added,NaN,NaN,NaN
133,Acacia lachnocarpa,Woolly-fruited Wattle,Acacia lachnocarpa,Priority 1: Poorly-known species,added,NaN,NaN,NaN
218,Acacia sp. Forrestania (D. Angus DA 3001),None,Acacia sp. Forrestania (D.Angus DA 3001),Priority 1: Poorly-known species,added,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4047,Latrobea colophona,NaN,NaN,NaN,removed,None,Latrobea colophona,Critically Endangered
4057,Hibbertia sp. Toolbrunup (J.R. Wheeler 2504),NaN,NaN,NaN,removed,None,Hibbertia sp. Toolbrunup (J.R.Wheeler 2504),Critically Endangered
4074,"Leucopogon sp. Flynn (F. Hort, J. Hort & A. Lo...",NaN,NaN,NaN,removed,None,"Leucopogon sp. Flynn (F.Hort, J.Hort & A.Lowri...",Critically Endangered
4112,Cyphanthera odgersii subsp. occidentalis,NaN,NaN,NaN,removed,None,Cyphanthera odgersii subsp. occidentalis,Critically Endangered


In [49]:
# %%script echo skipping # comment this line to do a list check between test and production and save the changelog
filename = "WA-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr467" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr18406" + "?max=10000&includeKVP=true"
changelist = lf.get_changelist(testListUrl, prodListUrl )
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

,name,commonName_new,scientificName_new,status_new,listUpdate,commonName_old,scientificName_old,status_old
0,Abildgaardia pachyptera,None,Abildgaardia pachyptera,Priority 1: Poorly-known species,added,NaN,NaN,NaN
14,Acacia ancistrophylla var. lissophylla,Dwarf Myall,Acacia ancistrophylla var. lissophylla,Priority 2: Poorly-known species,added,NaN,NaN,NaN
60,Acacia corusca,Shimmer Wattle,Acacia corusca,Priority 1: Poorly-known species,added,NaN,NaN,NaN
126,Acacia karinae,Mulga,Acacia,Priority 1: Poorly-known species,added,NaN,NaN,NaN
133,Acacia lachnocarpa,Woolly-fruited Wattle,Acacia lachnocarpa,Priority 1: Poorly-known species,added,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4024,Utricularia sp. Theda (M.D. Barrett MDB 2056),NaN,NaN,NaN,removed,None,Utricularia sp. Theda (M.D.Barrett MDB 2056),Priority 1: Poorly-known species
4031,Velleia exigua,NaN,NaN,NaN,removed,None,Velleia exigua,Priority 2: Poorly-known species
4032,Verreauxia verreauxii,NaN,NaN,NaN,removed,Spindly Verreauxia,Verreauxia verreauxii,"Priority 4: Rare, Near Threatened and other sp..."
4099,Verticordia staminosa var. cylindracea,NaN,NaN,NaN,removed,None,Verticordia staminosa var. cylindracea,Vulnerable


In [48]:
# %%script echo skipping # comment this line to save the prod list to the historical directory

filename = "WA-conservation.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr2201" + "?max=10000&includeKVP=true"
prodList = lf.download_ala_list(prodListUrl) # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
filename = "WA-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr467" + "?max=10000&includeKVP=true"
prodList = lf.download_ala_list(prodListUrl) # save the prod list to the historical lists directory
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)